# Download the Database of Regulatory Elements

In [1]:
%load_ext rpy2.ipython

In [2]:
%%bash
wget -O allGeneCorrelations100000.p05_v3.txt.gz http://big.databio.org/RED/allGeneCorrelations100000.p05_v3.txt.gz
zcat allGeneCorrelations100000.p05_v3.txt.gz|cut -f8,10|grep '^MIR\|^hsa-mir'|uniq|sort|uniq > allGeneCorrelations100000.p05_v3.MIR.ENSG_IDs.txt
head <(zcat allGeneCorrelations100000.p05_v3.txt.gz)
head allGeneCorrelations100000.p05_v3.MIR.ENSG_IDs.txt

dhs_chr	dhs_start	dhs_end	dhs_id	gene_chr	gene_start	gene_end	gene_name	metaprobeset_id	ensemblID	cor	pval
chrX	99861860	99862010	2861605	chrX	99885757	99891766	TSPAN6	1	ENSG00000000003	-0.5445678	0.006
chrX	99870060	99870210	2861609	chrX	99885757	99891766	TSPAN6	1	ENSG00000000003	 0.3683153	0.993
chrX	99880960	99881110	2861619	chrX	99885757	99891766	TSPAN6	1	ENSG00000000003	-0.4026298	0.020
chrX	99891065	99891215	2861626	chrX	99885757	99891766	TSPAN6	1	ENSG00000000003	 0.2857770	0.988
chrX	99891260	99891410	2861627	chrX	99885757	99891766	TSPAN6	1	ENSG00000000003	 0.2643461	0.983
chrX	99891740	99891890	2861629	chrX	99885757	99891766	TSPAN6	1	ENSG00000000003	 0.4594339	0.998
chrX	99891900	99892050	2861630	chrX	99885757	99891766	TSPAN6	1	ENSG00000000003	 0.4258412	0.997
chrX	99899060	99899210	2861638	chrX	99885757	99891766	TSPAN6	1	ENSG00000000003	 0.4342573	0.998
chrX	99904080	99904230	2861647	chrX	99885757	99891766	TSPAN6	1	ENSG00000000003	 0.3240553	0.990
hsa-mir-220a	ENSG00000207655


--2017-03-17 18:28:39--  http://big.databio.org/RED/allGeneCorrelations100000.p05_v3.txt.gz
Resolving big.databio.org (big.databio.org)... 192.64.119.17
Connecting to big.databio.org (big.databio.org)|192.64.119.17|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: http://www.biomedical-sequencing.at/bocklab/nsheffield/RED/allGeneCorrelations100000.p05_v3.txt.gz [following]
--2017-03-17 18:28:39--  http://www.biomedical-sequencing.at/bocklab/nsheffield/RED/allGeneCorrelations100000.p05_v3.txt.gz
Resolving www.biomedical-sequencing.at (www.biomedical-sequencing.at)... 149.148.226.60
Connecting to www.biomedical-sequencing.at (www.biomedical-sequencing.at)|149.148.226.60|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.biomedical-sequencing.at/bocklab/nsheffield/RED/allGeneCorrelations100000.p05_v3.txt.gz [following]
--2017-03-17 18:28:40--  https://www.biomedical-sequencing.at/bocklab/nsheffield/RED/allGeneC

# Download TCGA-OV Data

In [3]:
%%R
#source("https://bioconductor.org/biocLite.R")
#biocLite("TCGAbiolinks")
#library("TCGAbiolinks")
#devtools::install_github(repo = "BioinformaticsFMRP/TCGAbiolinks")
library("TCGAbiolinks")
sessionInfo()

# Uses the example from 
# https://bioconductor.org/packages/release/bioc/vignettes/TCGAbiolinks/inst/doc/tcgaBiolinks.html#mirna-expression-data-downstream-analysis-brca
CancerProject <- "TCGA-OV"
DataDirectory <- paste0("GDC/",gsub("-","_",CancerProject))
FileNameData <- paste0(DataDirectory, "_","miRNA_gene_quantification",".rda")

query.miR <- GDCquery(project = CancerProject, 
                      data.category = "Gene expression",
                      data.type = "miRNA gene quantification",
                      file.type = "hg19.mirna",
                      legacy = TRUE)

samplesDown.miR <- query.miR$results[[1]]$cases

dataSmTP.miR <- TCGAquery_SampleTypes(barcode = samplesDown.miR, typesample = 'TP')

queryDown.miR <- GDCquery(project = CancerProject, 
                          data.category = "Gene expression",
                          data.type = "miRNA gene quantification",
                          file.type = "hg19.mirna",
                          legacy = TRUE,
                          barcode = c(dataSmTP.miR))

GDCdownload(query = queryDown.miR,
            directory = DataDirectory)

dataAssy.miR <- GDCprepare(query = queryDown.miR, 
                           save = TRUE, 
                           save.filename = FileNameData, 
                           summarizedExperiment = TRUE,
                           directory =DataDirectory )
rownames(dataAssy.miR) <- dataAssy.miR$miRNA_ID

# using read_counts data 
#read_countData <-  colnames(dataAssy.miR)[grep("count", colnames(dataAssy.miR))]
#dataAssy.miR <- dataAssy.miR[,read_countData]
#colnames(dataAssy.miR) <- gsub("read_count_","", colnames(dataAssy.miR))

# using normalized reads per million data
read_rpmData <-  colnames(dataAssy.miR)[grep("reads_per_million_miRNA_mapped", colnames(dataAssy.miR))]
dataAssy.miR <- dataAssy.miR[,read_rpmData]
colnames(dataAssy.miR) <- gsub("reads_per_million_miRNA_mapped_","", colnames(dataAssy.miR))

#write.table(data.frame("miRNA"=rownames(dataAssy.miR),dataAssy.miR),"miRNA_quantifications.txt", row.names=FALSE)
#dataAssy.miR.transposed = t(dataAssy.miR)
#write.table(data.frame("TCGA_barcode"=rownames(dataAssy.miR.transposed),dataAssy.miR.transposed),"miRNA_quantifications_transposed.txt", row.names=FALSE)

mirna.rpm <- dataAssy.miR
#rownames(mirna.rpm) <-gsub("(?<=[A-Z])-","",gsub("HSA-LET-","MIRLET",gsub("HSA-MIR-","MIR",toupper(rownames(mirna.rpm)))), perl=TRUE)
write.table(data.frame("miRNA"=rownames(mirna.rpm),mirna.rpm),"miRNA_quantifications.reads_per_million_miRNA_mapped.txt", row.names=FALSE, sep = "\t")
mirna.rpm.transposed <- data.frame(t(mirna.rpm))

patient_barcode_from_sample_barcode <- function (sample_barcode) {
  return(paste(strsplit(sample_barcode,"-")[[1]][1:3], collapse="-")[1])
}

#rownames(mirna.rpm.transposed) <- lapply(rownames(mirna.rpm.transposed), patient_barcode_from_sample_barcode)
#mirna.rpm.transposed$bcr_patient_barcode <- rownames(mirna.rpm.transposed)
mirna.rpm.transposed$bcr_patient_barcode <- sapply(rownames(mirna.rpm.transposed), patient_barcode_from_sample_barcode)
write.table(data.frame("miRNA"=rownames(mirna.rpm.transposed),mirna.rpm.transposed),"miRNA_quantifications.reads_per_million_miRNA_mapped.transposed.txt", row.names=FALSE, sep = "\t")

clin.query <- GDCquery(project = "TCGA-OV", data.category = "Clinical")
json  <- tryCatch(GDCdownload(clin.query), 
                  error = function(e) GDCdownload(clin.query, method = "client"))
clinical.patient <- GDCprepare_clinic(clin.query, clinical.info = "patient")
clinical.patient.followup <- GDCprepare_clinic(clin.query, clinical.info = "follow_up")
clinical.patient.drug <- GDCprepare_clinic(clin.query, clinical.info = "drug")
clinical.patient.radiation <- GDCprepare_clinic(clin.query, clinical.info = "radiation")
clinical.patient.new_tumor_event <- GDCprepare_clinic(clin.query, clinical.info = "new_tumor_event")
clinical.patient.admin <- GDCprepare_clinic(clin.query, clinical.info = "admin")
clinical.patient.stage_event <- GDCprepare_clinic(clin.query, clinical.info = "stage_event")
clinical.index <- GDCquery_clinic("TCGA-OV")
write.table(clinical.patient,"clinical.patient.txt", row.names=FALSE, sep = "\t")

colnames(clinical.patient.followup) <- gsub("form_completion","form_completion_followup",colnames(clinical.patient.followup))
write.table(clinical.patient.followup,"clinical.patient.followup.txt", row.names=FALSE, sep = "\t")

colnames(clinical.patient.drug) <- gsub("form_completion","form_completion_drug",colnames(clinical.patient.drug))
write.table(clinical.patient.drug,"clinical.patient.drug.txt", row.names=FALSE, sep = "\t")

colnames(clinical.patient.radiation) <- gsub("form_completion","form_completion_radiation",colnames(clinical.patient.new_tumor_event))
write.table(clinical.patient.radiation,"clinical.patient.radiation.txt", row.names=FALSE, sep = "\t")

colnames(clinical.patient.new_tumor_event) <- gsub("form_completion","form_completion_new_tumor_event",colnames(clinical.patient.new_tumor_event))
write.table(clinical.patient.new_tumor_event,"clinical.patient.new_tumor_event.txt", row.names=FALSE, sep = "\t")

colnames(clinical.patient.stage_event) <- gsub("form_completion","form_completion_stage_event",colnames(clinical.patient.stage_event))
write.table(clinical.patient.stage_event,"clinical.patient.stage_event.txt", row.names=FALSE, sep = "\t")

colnames(clinical.patient.admin) <- gsub("form_completion","form_completion_admin",colnames(clinical.patient.admin))
write.table(clinical.patient.admin,"clinical.patient.admin.txt", row.names=FALSE, sep = "\t")

clinical.patient <- merge(clinical.patient,clinical.patient.drug, all=TRUE, by = "bcr_patient_barcode")

clinical.patient <- merge(clinical.patient,clinical.patient.followup, all=TRUE, by = "bcr_patient_barcode")

clinical.patient <- merge(clinical.patient,clinical.patient.admin, all=TRUE, by = "bcr_patient_barcode")

clinical.patient <- merge(clinical.patient,clinical.patient.stage_event, all=TRUE, by = "bcr_patient_barcode")

#clinical.patient <- merge(clinical.patient,clinical.patient.radiation, all=TRUE, by = "bcr_patient_barcode")

clinical.patient <- merge(clinical.patient,clinical.patient.new_tumor_event, all=TRUE, by = "bcr_patient_barcode")

write.table(clinical.patient,"clinical.patient.drug.followup.admin.stage_event.new_tumor_event.txt", row.names=FALSE, sep = "\t")

clinical.patient.dhs_mirna_rpm <- clinical.patient

clinical.patient.mirna_rpm <- merge(clinical.patient, mirna.rpm.transposed, all=TRUE, by = "bcr_patient_barcode")

write.table(clinical.patient.mirna_rpm,"clinical.patient.drug.followup.admin.stage_event.new_tumor_event.mirna_rpm.txt", row.names=FALSE, sep = "\t")

dhs.ensg_ids <- read.table("allGeneCorrelations100000.p05_v3.MIR.ENSG_IDs.txt",stringsAsFactors = FALSE)
ensembl_like_names <- gsub("(?<=[A-Z])-","",gsub("HSA-LET-","MIRLET",gsub("HSA-MIR-","MIR",toupper(rownames(mirna.rpm)))), perl=TRUE)
dhs.mirna.rpm <- mirna.rpm[ensembl_like_names %in% intersect(ensembl_like_names, dhs.ensg_ids[,1]),]
dhs.mirna.rpm.transposed <- data.frame(t(dhs.mirna.rpm))
dhs.mirna.rpm.transposed$bcr_patient_barcode <- sapply(rownames(dhs.mirna.rpm.transposed), patient_barcode_from_sample_barcode)
write.table(data.frame("miRNA"=rownames(dhs.mirna.rpm.transposed),dhs.mirna.rpm.transposed),"DHS_subset.miRNA_quantifications.reads_per_million_miRNA_mapped.transposed.txt", row.names=FALSE, sep = "\t")

clinical.patient.dhs_mirna_rpm <- merge(clinical.patient.dhs_mirna_rpm, dhs.mirna.rpm.transposed, all=TRUE, by = "bcr_patient_barcode")
write.table(clinical.patient.dhs_mirna_rpm,"clinical.patient.drug.followup.admin.stage_event.new_tumor_event.dhs_subset_mirna_rpm.txt", row.names=FALSE, sep = "\t")


/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: ##############################################################################
Pathview is an open source software package distributed under GNU General
Public License version 3 (GPLv3). Details of GPLv3 is available at
http://www.gnu.org/licenses/gpl-3.0.html. Particullary, users are required to
formally cite the original Pathview paper (not just mention it) in publications
or products. For details, do citation("pathview") within R.

The pathview downloads and uses KEGG data. Non-academic uses may require a KEGG
license agreement (details at http://www.kegg.jp/kegg/legal.html).
##############################################################################

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:185: RRuntimeWa

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |===================================================================== |  98%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


# Get DHS to MIR Lookup

In [4]:
%%bash
zcat allGeneCorrelations100000.p05_v3.txt.gz|awk '{print $8"\t"$1":"$2"-"$3}'|grep '^MIR'|sort|uniq|sort \
> MIRNA_to_DHS_Region.txt
head MIRNA_to_DHS_Region.txt

MIR100	chr11:121933580-121933730
MIR100	chr11:121947520-121947670
MIR100	chr11:121953060-121953210
MIR100	chr11:121955100-121955250
MIR100	chr11:121956080-121956230
MIR100	chr11:121959860-121960010
MIR100	chr11:121962720-121962870
MIR100	chr11:121963520-121963670
MIR100	chr11:121964560-121964710
MIR100	chr11:121966780-121966930


# Reformat so that a DHS region has only one line with all related genes

In [5]:
%%R
MIRNA_to_DHS_Region <- read.table('MIRNA_to_DHS_Region.txt', stringsAsFactors = FALSE, 
                                  col.names = c("Gene","DHS_Region"))
head(MIRNA_to_DHS_Region)

    Gene                DHS_Region
1 MIR100 chr11:121933580-121933730
2 MIR100 chr11:121947520-121947670
3 MIR100 chr11:121953060-121953210
4 MIR100 chr11:121955100-121955250
5 MIR100 chr11:121956080-121956230
6 MIR100 chr11:121959860-121960010


In [6]:
%%R
MIRNA_to_DHS_Region.aggregate <- aggregate(Gene ~ DHS_Region, data = MIRNA_to_DHS_Region, paste, collapse = ' ')
colnames(MIRNA_to_DHS_Region.aggregate) <- c("DHS_Region", "Genes")
head(MIRNA_to_DHS_Region.aggregate)
write.table(MIRNA_to_DHS_Region.aggregate,"MIRNA_to_DHS_Region.aggregate.txt",sep="\t",row.names=FALSE,quote=FALSE)

In [7]:
%%bash
head MIRNA_to_DHS_Region.aggregate.txt

DHS_Region	Genes
chr10:100055780-100055930	MIR1287
chr10:100062800-100062950	MIR1287
chr10:100069885-100070035	MIR1287
chr10:100074460-100074610	MIR1287
chr10:100078720-100078870	MIR1287
chr10:100083120-100083270	MIR1287
chr10:100098180-100098330	MIR1287
chr10:100098700-100098850	MIR1287
chr10:100104360-100104510	MIR1287


In [8]:
%%bash
cat MIRNA_to_DHS_Region.aggregate.txt|cut -f1|tail -n +2|sort|uniq|sort|sed 's|:|\t|'|sed 's|-|\t|' \
|awk 'BEGIN{FS=OFS="\t"}{print $1,$2-1,$3,$1":"$2"-"$3}'|awk '{print $0":+\t0\t+";print $0":-\t1\t-";}' \
|sort -k1,1 -k2,2n \
> MIRNA_to_DHS_Region.aggregate.bothstrands.bed
head MIRNA_to_DHS_Region.aggregate.bothstrands.bed
cat MIRNA_to_DHS_Region.aggregate.bothstrands.bed|awk '$6=="+"' \
> MIRNA_to_DHS_Region.aggregate.plus_strand.bed

chr1	1003204	1003355	chr1:1003205-1003355:+	0	+
chr1	1003204	1003355	chr1:1003205-1003355:-	1	-
chr1	1004124	1004275	chr1:1004125-1004275:+	0	+
chr1	1004124	1004275	chr1:1004125-1004275:-	1	-
chr1	1004284	1004435	chr1:1004285-1004435:+	0	+
chr1	1004284	1004435	chr1:1004285-1004435:-	1	-
chr1	1004524	1004675	chr1:1004525-1004675:+	0	+
chr1	1004524	1004675	chr1:1004525-1004675:-	1	-
chr1	1004684	1004835	chr1:1004685-1004835:+	0	+
chr1	1004684	1004835	chr1:1004685-1004835:-	1	-


# Retrieve hg19 human genome for sequences

In [9]:
%%bash
>hg19.fa
hg19_url_prefix="ftp://hgdownload.cse.ucsc.edu/goldenPath/hg19/chromosomes/"
for chr_num in $(seq 22) M X Y
do
    hg19_chr_filename="chr""$chr_num"".fa.gz"
    wget -O "$hg19_chr_filename" "$hg19_url_prefix""$hg19_chr_filename"
    zcat "$hg19_chr_filename" >> hg19.fa
done
ls hg19.fa

hg19.fa


--2017-03-17 18:32:45--  ftp://hgdownload.cse.ucsc.edu/goldenPath/hg19/chromosomes/chr1.fa.gz
           => ‘chr1.fa.gz’
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /goldenPath/hg19/chromosomes ... done.
==> SIZE chr1.fa.gz ... 73773666
==> PASV ... done.    ==> RETR chr1.fa.gz ... done.
Length: 73773666 (70M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  0%  859K 84s
    50K .......... .......... .......... .......... ..........  0% 1012K 77s
   100K .......... .......... .......... .......... ..........  0% 1.49M 67s
   150K .......... .......... .......... .......... ..........  0% 1.98M 59s
   200K .......... .......... .......... .......... ..........  0% 2.03M 54s
   250K .......... .......... ........

In [10]:
%%bash
#http://some-hints.blogspot.com/2013/07/how-to-convert-fasta-file-to.html
rm chr*.fa.gz
cat hg19.fa|awk '{ if ($0 !~ />/) {print toupper($0)} else {print $0} }' \
>hg19.upper.fa
samtools faidx hg19.upper.fa
rm hg19.fa

# Get gene names of important genes

In [11]:
%%bash
cp kevin/out/* .

In [12]:
%%R
get_important_mirna_names <- function(gene_importance_file_path, output_important_gene_names_file_path){
  gene_importance <- read.csv(gene_importance_file_path, row.names = 1)[,1,FALSE]
  gene_importance_threshold <- abs(min(0, min(gene_importance)))
  gene_importance_threshold
  important_genes <- gene_importance[which(gene_importance > gene_importance_threshold), 1, FALSE]
  important_genes
  important_gene_names <- rownames(important_genes)
  important_gene_names
  important_gene_ensembl_like_names <- gsub("(?<=[A-Z])-","",gsub("HSA-LET-","MIRLET",gsub("HSA-MIR-","MIR",toupper(gsub("\\.","-",(important_gene_names))))), perl=TRUE)
  important_gene_ensembl_like_names
  cat(important_gene_ensembl_like_names , file = output_important_gene_names_file_path, sep = "\n", fill = FALSE, labels = NULL,append = FALSE)
  return(important_gene_ensembl_like_names)
}


In [13]:
%%R
get_important_mirna_names("os_gene_importance.csv", "os_gene_importance.important_genes.txt")
get_important_mirna_names("stage_gene_importance.csv", "stage_gene_importance.important_genes.txt")
get_important_mirna_names("pfs_gene_importance.csv", "pfs_gene_importance.important_genes.txt")


 [1] "MIRLET7B" "MIR101-1" "MIR107"   "MIR10A"   "MIR124-2" "MIR125B1"
 [7] "MIR130B"  "MIR134"   "MIR135A1" "MIR138-1" "MIR141"   "MIR143"  
[13] "MIR16-2"  "MIR187"   "MIR18A"   "MIR194-1" "MIR200A"  "MIR200C" 
[19] "MIR218-1" "MIR22"    "MIR221"   "MIR222"   "MIR223"   "MIR30A"  
[25] "MIR30C2"  "MIR30D"   "MIR30E"   "MIR323"   "MIR543"   "MIR877"  
[31] "MIR99A"  


In [14]:
%%bash
wc -l "os_gene_importance.important_genes.txt"
cat "os_gene_importance.important_genes.txt"
wc -l "stage_gene_importance.important_genes.txt"
cat "stage_gene_importance.important_genes.txt"
wc -l "pfs_gene_importance.important_genes.txt"
cat "pfs_gene_importance.important_genes.txt"
cat "os_gene_importance.important_genes.txt" "stage_gene_importance.important_genes.txt" "pfs_gene_importance.important_genes.txt" \
|sort|uniq \
> superset.important_genes.txt
wc -l superset.important_genes.txt
cat superset.important_genes.txt
cp superset.important_genes.txt superset_gene_importance.important_genes.txt

14 os_gene_importance.important_genes.txt
MIRLET7A2
MIRLET7B
MIR125A
MIR130B
MIR141
MIR143
MIR187
MIR18A
MIR191
MIR203
MIR219-1
MIR24-2
MIR877
MIR98
50 stage_gene_importance.important_genes.txt
MIRLET7C
MIRLET7E
MIRLET7G
MIRLET7I
MIR101-1
MIR107
MIR126
MIR127
MIR1287
MIR132
MIR134
MIR140
MIR141
MIR142
MIR150
MIR152
MIR155
MIR16-2
MIR17
MIR184
MIR191
MIR196A1
MIR199A1
MIR199A2
MIR19A
MIR203
MIR204
MIR20A
MIR21
MIR218-2
MIR219-1
MIR221
MIR222
MIR25
MIR27A
MIR299
MIR30A
MIR30C1
MIR30E
MIR323
MIR33A
MIR34A
MIR34C
MIR487A
MIR543
MIR7-1
MIR877
MIR93
MIR99A
MIR99B
31 pfs_gene_importance.important_genes.txt
MIRLET7B
MIR101-1
MIR107
MIR10A
MIR124-2
MIR125B1
MIR130B
MIR134
MIR135A1
MIR138-1
MIR141
MIR143
MIR16-2
MIR187
MIR18A
MIR194-1
MIR200A
MIR200C
MIR218-1
MIR22
MIR221
MIR222
MIR223
MIR30A
MIR30C2
MIR30D
MIR30E
MIR323
MIR543
MIR877
MIR99A
72 superset.important_genes.txt
MIR101-1
MIR107
MIR10A
MIR124-2
MIR125A
MIR125B1
MIR126
MIR127
MIR1287
MIR130B
MIR132
MIR134
MIR135A1
MIR138-1
MIR140
MIR141

# Get BED of DHS of MIRNA

In [15]:
%%bash
cat MIRNA_to_DHS_Region.aggregate.txt \
|tail -n +2 \
|sort|uniq|sort \
|sed 's|:|\t|'|sed 's|-|\t|' \
|awk 'BEGIN{FS=OFS="\t"}{print $1,$2-1,$3,$1":"$2"-"$3":+:"$4"::"}' \
|sed 's| |::|g' \
|sort -k1,1 -k2,2n|uniq \
> MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed


# Get FASTA of DHS of MIRNA

In [16]:
%%bash
bedtools getfasta \
-fi hg19.upper.fa \
-bed MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed \
-s -name \
-fo MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.fa


# Download motif database

In [17]:
%%bash
wget -O motif_databases.12.15.tar.gz http://meme-suite.org/meme-software/Databases/motifs/motif_databases.12.15.tgz
tar -zxvf motif_databases.12.15.tar.gz

motif_databases/
motif_databases/ARABD/
motif_databases/CIS-BP/
motif_databases/CISBP-RNA/
motif_databases/ECOLI/
motif_databases/EUKARYOTE/
motif_databases/FLY/
motif_databases/HUMAN/
motif_databases/JASPAR/
motif_databases/log.txt
motif_databases/MALARIA/
motif_databases/MIRBASE/
motif_databases/motif_db.csv
motif_databases/MOUSE/
motif_databases/PROKARYOTE/
motif_databases/RNA/
motif_databases/TFBSshape/
motif_databases/tmp
motif_databases/WORM/
motif_databases/YEAST/
motif_databases/YEAST/macisaac_yeast.v1.meme
motif_databases/YEAST/scpd_matrix.meme
motif_databases/YEAST/SwissRegulon_s_cer.meme
motif_databases/YEAST/yeast_uniprobe_GR09.meme
motif_databases/YEAST/YEASTRACT_20130918.meme
motif_databases/WORM/uniprobe_worm.meme
motif_databases/TFBSshape/TFBSshape_JASPAR.meme
motif_databases/TFBSshape/TFBSshape_UniPROBE.meme
motif_databases/RNA/Ray2013_rbp_All_Species.dna_encoded.meme
motif_databases/RNA/Ray2013_rbp_All_Species.meme
motif_databases/RNA/Ray2013_rbp_Arabidopsis_thaliana.

--2017-03-17 18:36:52--  http://meme-suite.org/meme-software/Databases/motifs/motif_databases.12.15.tgz
Resolving meme-suite.org (meme-suite.org)... 54.68.135.202
Connecting to meme-suite.org (meme-suite.org)|54.68.135.202|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12563298 (12M)
Saving to: ‘motif_databases.12.15.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  612K 20s
    50K .......... .......... .......... .......... ..........  0% 1.23M 15s
   100K .......... .......... .......... .......... ..........  1% 3.67M 11s
   150K .......... .......... .......... .......... ..........  1% 10.3M 8s
   200K .......... .......... .......... .......... ..........  2% 1.77M 8s
   250K .......... .......... .......... .......... ..........  2% 8.55M 7s
   300K .......... .......... .......... .......... ..........  2% 8.22M 6s
   350K .......... .......... .......... .......... ..........  3% 8.52M 5s
   400K .......... .......... .....

# AME using all DHSs as background and HOMER using DHSs not associated with important miRNAs as background

First, we retrieve sequences of all DHSs to derive a Markov Model used to determine the likelihood of a motif being found in non-sites within the DHSs.
Then, we retrieve the sequences of all DHSs associated with the important miRNAs. This is the primary query sequences set.
Then, we retrieve the sequences of all DHSs not associated with the important miRNAS. This is the control sequence set.
AME is used to determine if a motif is enriched in the primary query sequence set, over the control sequence set. AME uses a Fisher Exact test by default, and has the option of a non-'proper' heuristic implementation of the Wilcoxon rank-sum test (or U test). Both tests are one-tailed for higher enrichment in the primary query set. Both options for testing are used.

For HOMER, we use sequences of all DHSs associated with the important miRNAs as input, with sequences of all DHSs not associated with the important miRNAS as control.

In [18]:
%%bash
# Get DHS intervals
zcat allGeneCorrelations100000.p05_v3.txt.gz|awk '{print $1"\t"$2-1"\t"$3"\tDHS|"$1":"$2"-"$3"\t0\t+"}'|awk 'NR!=1' \
|uniq \
|sort -k1,1 -k2,2n \
|uniq \
> DHS_sequences.plus_strand.bed

# Get sequences of DHS intervals
bedtools getfasta -fi hg19.upper.fa -bed DHS_sequences.plus_strand.bed -s -name \
-fo DHS_sequences.plus_strand.hg19.upper.fa

# Get background model
fasta-get-markov -m 3 \
-dna DHS_sequences.plus_strand.hg19.upper.fa \
DHS_sequences.plus_strand.hg19.upper.markov_background

# Perform vertebrate transcript factor motif enrichment
function get_motifs_enriched_in_important_genes_DHSs() {
echo "Analyzing important gene list:"
important_gene_list=$1
echo "$important_gene_list"

# Get DHS of important microRNAs
cat MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed \
|grep -Fwf \
<(cat "$important_gene_list"|awk '{print ":"$1":"}') \
> $(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed

# Get DHS of microRNAs not associated with important miRNAs
cat MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed \
|grep -vFwf \
<(cat "$important_gene_list"|awk '{print ":"$1":"}') \
> $(basename "$important_gene_list").control.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed

# Get sequences from DHS intervals
bedtools getfasta \
-fi hg19.upper.fa \
-bed $(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed \
-s -name \
-fo $(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.fa

bedtools getfasta \
-fi hg19.upper.fa \
-bed $(basename "$important_gene_list").control.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed \
-s -name \
-fo $(basename "$important_gene_list").control.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.fa

# Perform Analysis of Motif Enrichment (AME)
ame \
--bgformat 2 \
--bgfile DHS_sequences.plus_strand.hg19.upper.markov_background \
--control $(basename "$important_gene_list").control.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.fa \
--oc $(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.ame_with_control_and_all_DHS_background_out \
$(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.fa \
motif_databases/JASPAR/JASPAR_CORE_2016_vertebrates.meme

# Perform Analysis of Motif Enrichment (AME) with ranksum test
ame \
--bgformat 2 \
--method ranksum \
--bgfile DHS_sequences.plus_strand.hg19.upper.markov_background \
--control $(basename "$important_gene_list").control.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.fa \
--oc $(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.ame_with_control_and_all_DHS_background_out.ranksum \
$(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.fa \
motif_databases/JASPAR/JASPAR_CORE_2016_vertebrates.meme


echo "AME Results In:"
echo $(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.ame_with_control_and_all_DHS_background_out/ame.html

echo "AME Results In:"
echo $(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.ame_with_control_and_all_DHS_background_out.ranksum/ame.html

# Corroborate results with HOMER
findMotifsGenome.pl \
$(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed \
hg19 \
$(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.homer_with_control_as_background.hg \
-size given \
-preparse \
-mset vertebrates \
-bg $(basename "$important_gene_list").control.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed \
-h

echo $(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.homer_with_control_as_background.hg/knownResults.html

#findMotifsGenome.pl \
#$(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed \
#hg19 \
#$(basename "$important_gene_list").MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.homer_with_all_DHS_as_background.hg \
#-size given \
#-preparse \
#-mset vertebrates \
#-bg DHS_sequences.plus_strand.bed \
#-h
}

get_motifs_enriched_in_important_genes_DHSs "os_gene_importance.important_genes.txt"

get_motifs_enriched_in_important_genes_DHSs "stage_gene_importance.important_genes.txt"

get_motifs_enriched_in_important_genes_DHSs "pfs_gene_importance.important_genes.txt"

get_motifs_enriched_in_important_genes_DHSs "superset_gene_importance.important_genes.txt"

538647 151 431 151.3 81478057
Analyzing important gene list:
os_gene_importance.important_genes.txt
AME Results In:
os_gene_importance.important_genes.txt.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.ame_with_control_and_all_DHS_background_out/ame.html
AME Results In:
os_gene_importance.important_genes.txt.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.ame_with_control_and_all_DHS_background_out.ranksum/ame.html
os_gene_importance.important_genes.txt.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.homer_with_control_as_background.hg/knownResults.html
Analyzing important gene list:
stage_gene_importance.important_genes.txt
AME Results In:
stage_gene_importance.important_genes.txt.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.ame_with_control_and_all_DHS_background_out/ame.html
AME Results In:
stage_gene_importance.important_genes.txt.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.ame_wit

Writing results to output directory 'os_gene_importance.important_genes.txt.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.ame_with_control_and_all_DHS_background_out'.
Writing results to output directory 'os_gene_importance.important_genes.txt.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.hg19.upper.ame_with_control_and_all_DHS_background_out.ranksum'.

	Position file = os_gene_importance.important_genes.txt.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed
	Genome = hg19
	Output Directory = os_gene_importance.important_genes.txt.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.homer_with_control_as_background.hg
	Using actual sizes of regions (-size given)
	Fragment size set to given
	background position file: os_gene_importance.important_genes.txt.control.MIRNA_to_DHS_Region.aggregate.MIRNA_DHS_named.plus_strand.bed
	Using hypergeometric distribution for p-values
	Peak/BED file conversion summary:
		BED/Header formatted lines: 3